In [25]:
import pandas as pd

df = pd.read_csv('/content/mycocktail_UTF-8.csv')
df.head()

,Chinese_Name,English_Name,Ingredient1,Ingredient2,Ingredient3,Ingredient4,Ingredient5,Ingredient6,Ingredient7,Ingredient8
0,白色俄羅斯,White Russian,伏特加,咖啡香甜酒,鮮奶,NaN,NaN,NaN,NaN,NaN
1,瑪格麗特,Margarita,龍舌蘭,君度橙酒,檸檬汁,糖漿,NaN,NaN,NaN,NaN
2,黛綺莉,Daiquiri,蘭姆酒,檸檬汁,糖漿,NaN,NaN,NaN,NaN,NaN
3,自由古巴,Cuba Libre,蘭姆酒,可樂,檸檬角,NaN,NaN,NaN,NaN,NaN
4,藍色夏威夷,Blue Hawaii,蘭姆酒,伏特加,藍柑橘香甜酒,鳳梨汁,檸檬汁,糖漿,NaN,NaN


In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Chinese_Name  60 non-null     object
 1   English_Name  60 non-null     object
 2   Ingredient1   60 non-null     object
 3   Ingredient2   60 non-null     object
 4   Ingredient3   49 non-null     object
 5   Ingredient4   31 non-null     object
 6   Ingredient5   14 non-null     object
 7   Ingredient6   4 non-null      object
 8   Ingredient7   1 non-null      object
 9   Ingredient8   1 non-null      object
dtypes: object(10)
memory usage: 4.8+ KB


In [27]:
# combine all strings:
df2 = df.drop(['English_Name'],axis=1)

df2['data'] = df2[df2.columns[1:]].apply(
    lambda x: ' '.join(x.dropna().astype(str)),
    axis=1
)

print(df2['data'].head())

0                 伏特加 咖啡香甜酒 鮮奶
1              龍舌蘭 君度橙酒 檸檬汁 糖漿
2                   蘭姆酒 檸檬汁 糖漿
3                   蘭姆酒 可樂 檸檬角
4    蘭姆酒 伏特加 藍柑橘香甜酒 鳳梨汁 檸檬汁 糖漿
Name: data, dtype: object


In [28]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
vectorized = vectorizer.fit_transform(df2['data'])

In [29]:
from sklearn.metrics.pairwise import cosine_similarity

similarities = cosine_similarity(vectorized)

In [30]:
print(similarities)

[[1.         0.         0.         ... 0.         0.         0.        ]
 [0.         1.         0.57735027 ... 0.         0.5        0.5       ]
 [0.         0.57735027 1.         ... 0.         0.57735027 0.8660254 ]
 ...
 [0.         0.         0.         ... 1.         0.         0.        ]
 [0.         0.5        0.57735027 ... 0.         1.         0.5       ]
 [0.         0.5        0.8660254  ... 0.         0.5        1.        ]]


In [31]:
df = pd.DataFrame(similarities, columns=df['Chinese_Name'], index=df['Chinese_Name']).reset_index()

df.head()

Chinese_Name,Chinese_Name,白色俄羅斯,瑪格麗特,黛綺莉,自由古巴,藍色夏威夷,鳳梨可樂達,亞歷山大,側車,白蘭地亞歷山大,...,新鮮禁果,粉紅佳人,盤尼西林,蜜蜂之膝,肯巴利莫希托,帕洛瑪,藍色珊瑚礁,奧林匹克,伯爵茶馬丁尼,內華達
0,白色俄羅斯,1.000000,0.000000,0.000000,0.000000,0.235702,0.000000,0.0,0.000000,0.516398,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.0,0.000000,0.000000
1,瑪格麗特,0.000000,1.000000,0.577350,0.000000,0.408248,0.447214,0.0,0.750000,0.000000,...,0.288675,0.250000,0.250000,0.250000,0.447214,0.670820,0.288675,0.0,0.500000,0.500000
2,黛綺莉,0.000000,0.577350,1.000000,0.333333,0.707107,0.774597,0.0,0.577350,0.000000,...,0.333333,0.288675,0.288675,0.288675,0.516398,0.516398,0.333333,0.0,0.577350,0.866025
3,自由古巴,0.000000,0.000000,0.333333,1.000000,0.235702,0.258199,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.288675
4,藍色夏威夷,0.235702,0.408248,0.707107,0.235702,1.000000,0.730297,0.0,0.408248,0.182574,...,0.235702,0.204124,0.204124,0.204124,0.365148,0.365148,0.707107,0.0,0.408248,0.612372


In [32]:
input_cocktail = '柯夢波丹'
recommendations = pd.DataFrame(df.nlargest(5,input_cocktail)['Chinese_Name'])
recommendations = recommendations[recommendations['Chinese_Name']!=input_cocktail]
print(recommendations)

   Chinese_Name
47        神風特攻隊
1          瑪格麗特
7            側車
38       X.Y.Z.


In [33]:
import joblib

joblib.dump(df, 'cocktail_model.joblib')

['cocktail_model.joblib']

In [34]:
import pickle
pickle.dump(df.to_dict(),open('cocktail_dict.pkl','wb'))
pickle.dump(df,open('cocktail_similarity.pkl','wb'))